In [1]:
import warnings
warnings.filterwarnings("ignore", message=r"Passing", category=FutureWarning)
warnings.filterwarnings("ignore", message=r"Implicit", category=UserWarning)

In [2]:
import carla.models.catalog.load_model as loading_utils
import carla.models.catalog.train_model as training_utils
import json
import numpy as np
import os
import pandas as pd

from carla import log
from carla.models.negative_instances import predict_negative_instances
from carla.recourse_methods import Dice, Wachter
from carla import MLModelCatalog
from carla.data.catalog import DataCatalog
from copy import deepcopy
from datetime import datetime
from func_timeout import func_timeout, FunctionTimedOut
from ipynb.fs.full.metrics import current_MMD, disagreement_distance, measure_distribution, performance
from ipynb.fs.full.plotting import plot_distribution
from sklearn.model_selection import train_test_split
from typing import List

Using TensorFlow backend.


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [3]:
class DynamicCsvCatalog(DataCatalog):
    """
    Wrapper class for the DataCatalog similar to the built-in CsvCatalog but with new capabilities
    required to control data in the experiments.
    
    Attributes:
        file_path (str): 
            Path to the .csv file containing the dataset.
        categorical (List[str]): 
            Names of columns describing categorical features.
        continuous (List[str]):
            Names of columns describing continuous (i.e. numerical) features.
        immutables (List[str]):
            Names of columns describing immutable features, not supported by all generators.
        target (str):
            Name of the column that contains the target variable.
        test_size (float):
            Proportion of the dataset which should be withheld as an independent test set.
    """
    
    
    def __init__(self, file_path: str, categorical: List[str],  continuous: List[str],
                 immutables: List[str], target: str, test_size: float,
                 scaling_method: str = "MinMax", encoding_method: str = "OneHot_drop_binary",
                 positive=1, negative=0):
        
        self._categorical = categorical
        self._continuous = continuous
        self._immutables = immutables
        self._target = target
        self._positive = positive
        self._negative = negative

        # Load the raw data
        raw = pd.read_csv(file_path)
        train_raw, test_raw = train_test_split(raw, test_size=test_size)

        super().__init__("custom", raw, train_raw, test_raw,
                         scaling_method, encoding_method)

    @property
    def categorical(self) -> List[str]:
        return self._categorical

    @property
    def continuous(self) -> List[str]:
        return self._continuous

    @property
    def immutables(self) -> List[str]:
        return self._immutables

    @property
    def target(self) -> str:
        return self._target
    
    @property
    def positive(self) -> str:
        return self._positive
    
    @property
    def negative(self) -> str:
        return self._negative

In [4]:
class DynamicMLModelCatalog(MLModelCatalog):
    """
    Wrapper class for the MLModelCatalog that introduces additional functions
    allowing for the efficient and unbiased measurement of the dynamics of recourse.
    
    Attributes:
        data (DataCatalog):
            Dataset which will be used to train a model and conduct experiments.
        model_type (str):
            Black-box model used for classification, currently this class supports only ANNs and Logistic Regression.
        backend (str):
            Specifies the framework used on the backend, currently this class supports only PyTorch.
        cache (Boolean):
            If True, the framework will attempt to load a model that was previously cached.
        models_home (str):
            Path to the directory where models should be saved after they are trained.
        load_online: (Boolean):
            If True, a pretrained model will be loaded.
        kwargs (dict):
            Dictionary of optional keyworded arguments.
    """
    def __init__(self, data: DataCatalog, model_type: str, backend: str = "pytorch",
        cache: bool = True, models_home: str = None, load_online: bool = True, **kwargs) -> None:
        
        if backend != 'pytorch':
            raise NotImplementedError(f"Only PyTorch models are currently supported")
            
        if model_type not in ['ann', 'linear']:
            raise NotImplementedError(f"Model type not supported: {self.model_type}")
            
        save_name = model_type
        if model_type == "ann":
            save_name += f"_layers_{kwargs['save_name_params']}"
            
        self._save_name = save_name
        
        super().__init__(data, model_type, backend, cache,
                         models_home, load_online, **kwargs)
    
    @property
    def save_name(self) -> str:
        return self._save_name
        
    def params(self):        
        # Attempt to load the saved model
        self._model = loading_utils.load_trained_model(save_name=self._save_name,
                                                       data_name=self.data.name,
                                                       backend=self.backend)
        
        # This method should only be used when a model is already available
        if self._model is None:
            raise ValueError(f"No trained model found for {self._save_name}")
            
        for param in self._model.parameters():
            print(param)
        
    def retrain(self, learning_rate=0.01, epochs=5, batch_size=1, hidden_size=[4]):
        """
        Loads a cached model and retrains it on an updated dataset.
        
        Args:
            learning_rate (float):
                Size of the step at each epoch of the model training.
            epochs (int):
                Number of iterations of training.
            batch_size (int):
                Number of samples used at once in a gradient descent step, if '1' the procedure is stochastic.    
        """

        # Attempt to load the saved model
        self._model = loading_utils.load_trained_model(save_name=self._save_name,
                                                       data_name=self.data.name,
                                                       backend=self.backend)
        
        # This method should only be used when a model is already available
        if self._model is None:
            raise ValueError(f"No trained model found for {self._save_name}")
        
        # Sanity check to see if loaded model accuracy makes sense
        if self._model is not None:
            self._test_accuracy()
            
        # Get preprocessed data
        df_train = self.data.df_train
        df_test = self.data.df_test

        # All dataframes may have possibly changed
        x_train = df_train[list(set(df_train.columns) - {self.data.target})]
        y_train = df_train[self.data.target]
        x_test = df_test[list(set(df_test.columns) - {self.data.target})]
        y_test = df_test[self.data.target]

        # Order data (column-wise) before training
        x_train = self.get_ordered_features(x_train)
        x_test = self.get_ordered_features(x_test)
        
        log.info(f"Current balance: train set {y_train.mean()}, test set {y_test.mean()}")
        
        # Access the data in a format expected by PyTorch
        train_dataset = training_utils.DataFrameDataset(x_train, y_train)
        train_loader = training_utils.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_dataset = training_utils.DataFrameDataset(x_test, y_test)
        test_loader = training_utils.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

        # Retrain the model
        training_utils._training_torch(self._model, train_loader, test_loader,
                                       learning_rate, epochs)

        loading_utils.save_model(model=self._model, save_name=self._save_name,
                                 data_name=self.data.name, backend=self.backend)

In [ ]:
class RecourseGenerator():
    
    def __init__(self, name, dataset, model, recourse_method, params):
        self.name = name
        self.dataset = dataset
        self.model = model
        self.recourse_method = recourse_method
        self.params = params
        
        self.update_generator()
        
    def update_generator():
        self.generator = recourse_method(self.model, params)

In [5]:
# TODO: Generalize for other recourse generators
# TODO: What to do if a generator times out? do we accept different numbers of samples?
# TODO: Create a wrapper for RecourseMethod to allow for the measurement of multiple generators at once
class RecourseExperiment():
    """
    Allows to conduct an experiment about the dynamics of algorithmic recourse.
    
    Attributes:
        dataset (DataCatalog): 
            Catalog containing a dataframe, set of train and test records, and the target.
        generator_name (str):
            Name of the generator that is placed under test.
        generator_timeout (int): 
            Number of seconds after which the search for counterfactuals will time out.
        experiment_name (str):
            Name of the experiment that will be used as part of the directory name where results are saved.
        kwargs (dict):
            Dictionary of optional keyworded arguments.  
    """
    
    def __init__(self, dataset, total_recourse=0.5, recourse_per_epoch=0.01, generator_name='DICE',
                 positive_class=1, negative_class=0, generator_timeout=60, experiment_name='experiment', **kwargs):
    
        # Experiment data is saved into a new directory
        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
        self.experiment_name = f'{timestamp}_{experiment_name}'
        os.makedirs(f'../experiment_data/{self.experiment_name}')
        self.generator_name = generator_name
        self.generator_timeout = generator_timeout

        self.experiment_data = {
            self.generator_name: {0: {}},
            'wachter': {0: {}}
        }

        # Train a classifier on the dataset
        self.hyper_params = kwargs.get('hyper_params',
                                       {'learning_rate': 0.04, 'epochs': 3, 'batch_size': 1, 'hidden_size': [4]})
        self.dataset = dataset
        self.model = train_model(dataset, self.hyper_params)
        self.initial_model = deepcopy(self.model)

        # Recourse generated by DICE is compared with the Wachter generator, as they may modify data differently
        # we need to keep track of two models and two datasets and update them independently
        self.benchmark_dataset = deepcopy(dataset)
        self.benchmark_model = deepcopy(self.model)

        # factuals are a list of instances that the model expects to belong to the negative class;
        # in order to acurately measure the performance of the dataset we never change the test set
        self.factuals = predict_negative_instances(self.model, dataset.df_train)
        self.factuals_index = self.factuals.index.tolist()

        # Instantiate recourse generators
        self.dice_params = kwargs.get('generator_params', {"num": 1})
        self.benchmark_params = kwargs.get('benchmark_params', {"loss_type": "BCE", "t_max_min": 0.5})
        
        self.generator = Dice(self.model, self.dice_params)
        self.benchmark = Wachter(self.benchmark_model, self.benchmark_params)
        
        pos_individuals = dataset.df_train.loc[dataset.df_train['target'] == positive_class]
        self.initial_pos_sample = pos_individuals.sample(n=min(len(pos_individuals), 100)).to_numpy()
            
    def run(self, total_recourse=0.8, recourse_per_epoch=0.01):
        """
        Driver code to execute an experiment that allows to compare the dynamics of recourse 
        applied by some generator to a benchmark described by Wachter et al. (2017).
        
        Attributes:
            total_recourse (float): 
                Value between 0 and 1 representing the proportion of samples from the training set
                which should have recourse applied to them throughout the experiment.
            recourse_per_epoch (float): 
                Value between 0 and 1 representing the proportion of samples from the training set
                which should have recourse applied to them in a single iteration.
        """
    
        num_found_generator = 0
        num_found_benchmark = 0
           
        # Measure the data distribution and performance of models
        self.measure(0)
        
        # Plot initial data distributions
        path = f'../experiment_data/{self.experiment_name}'
        plot_distribution(self.dataset._df, self.model, path,
                  self.generator_name, 'distribution', 0)
        plot_distribution(self.benchmark_dataset._df, self.benchmark_model, path,
                          'wachter', 'distribution', 0)
        
        # Convert ratio of samples that should undergo recourse in a single epoch into a number
        recourse_per_epoch = max(int(recourse_per_epoch * len(self.factuals)), 1)
        # Convert ratio of samples that should undergo recourse in total into a number of epochs
        epochs = max(int(min(total_recourse, 1) * len(self.factuals) / recourse_per_epoch), 1)
                              
        for epoch in range(epochs - 1):
            log.info(f"Starting epoch {epoch + 1}")
            self.experiment_data[self.generator_name][epoch + 1] = {}
            self.experiment_data['wachter'][epoch + 1] = {}
                              
            # 4. Generate a subset S of factuals that have never been encountered by the generators
            sample_size = min(recourse_per_epoch, len(self.factuals_index))
            current_factuals_index = np.random.choice(self.factuals_index, replace=False, size=sample_size)
            current_test_factuals = self.dataset._df.iloc[current_factuals_index]
            current_benchmark_factuals = self.benchmark_dataset._df.iloc[current_factuals_index] 
                              
            # We do not want to accidentally generate a counterfactual from a counterfactual
            self.factuals_index = [f for f in self.factuals_index if f not in current_factuals_index]  

            # Apply recourse for S with DICE
            num_found_generator += self.apply_recourse_with_timeout(self.dataset,
                                                                    current_test_factuals,
                                                                    self.generator)

            # Update DICE model
            self.model = self.update_model(self.dataset, self.model,
                                           self.hyper_params, self.generator_name)


            # Apply recourse for S with Wachter
            num_found_benchmark += self.apply_recourse(self.benchmark_dataset,
                                                       current_benchmark_factuals,
                                                       self.benchmark)
            
            # Update the Wachter model
            self.benchmark_model = self.update_model(self.benchmark_dataset, self.benchmark_model,
                                                     self.hyper_params, 'Wachter')
                           
            # Measure the data distribution and performance of models
            self.measure(epoch + 1)

            # Plot data distributions
            plot_distribution(self.dataset._df, self.model, path,
                              self.generator_name, 'distribution', epoch + 1)
            plot_distribution(self.benchmark_dataset._df, self.benchmark_model, path,
                              'wachter', 'distribution', epoch + 1)
            
            self.generator = Dice(self.model, self.dice_params)
            self.benchmark = Wachter(self.benchmark_model, self.benchmark_params)

        # Measure the quality of recourse
        self.experiment_data['evaluation'] = {
            'dice': {
                'success_rate': num_found_generator / max(len(self.factuals.index) - len(self.factuals_index), 1)
            },
            'wachter': {
                'success_rate': num_found_benchmark / max(len(self.factuals.index) - len(self.factuals_index), 1)
            }
        }
        
    def measure(self, epoch):
        """
        Quantify the dataset and model and save into `experiment_data`.
        
        Args:
            epoch (int): 
                Current epoch in the experiment.
        """     
        # Measure the distributions of data
        self.experiment_data[self.generator_name][epoch]['distribution'] = measure_distribution(self.dataset)
        self.experiment_data['wachter'][epoch]['distribution'] = measure_distribution(self.benchmark_dataset)

        # Measure the current performance of models
        self.experiment_data[self.generator_name][epoch]["performance"] = performance(self.dataset,
                                                                         self.model)
        self.experiment_data['wachter'][epoch]["performance"] = performance(self.benchmark_dataset,
                                                                            self.benchmark_model)   
        
        # Measure the disagreement between current models and the initial model
        generator_disagreement = disagreement_distance(self.dataset._df_test, self.dataset.target,
                                                       self.initial_model, self.model)
        self.experiment_data[self.generator_name][epoch]['disagreement'] = generator_disagreement
        
        benchmark_disagreement = disagreement_distance(self.dataset._df_test, self.dataset.target,
                                                       self.initial_model, self.benchmark_model)
        self.experiment_data['wachter'][epoch]['disagreement'] = benchmark_disagreement
        
        print(f'benchmark {benchmark_disagreement} -- DICE {generator_disagreement}')
        
        # Measure the MMD
        self.experiment_data[self.generator_name][epoch]['MMD'] = current_MMD(self.dataset._df_train,
                                                                              self.dataset._positive,
                                                                              self.initial_pos_sample)
        
        self.experiment_data['wachter'][epoch]['MMD'] = current_MMD(self.benchmark_dataset._df_train,
                                                                    self.benchmark_dataset._positive,
                                                                    self.initial_pos_sample)
                         
            
    def update_model(self, dataset, model, hyper_params, generator_name):
        """
        Re-train the model based on an updated dataset
        
        Args:
            dataset (DataCatalog): 
                Catalog containing a dataframe, set of train and test records, and the target.
            model (MLModelCatalog) 
                Classifier with additional utilities required by CARLA.
            generator_name (str):
                Name of the generator that has its model updated.
                
        Returns:
            MLModelCatalog: A re-trained classifier.
        """
        # Ensure that the dataset saved by the model is always updated with counterfactuals
        model.data._df.update(dataset._df)
        model.data._df_train.update(dataset._df)
                              
        log.info(f'Updating the {generator_name} model')                      
        return train_model(dataset, hyper_params, retrain=True)                    
        
                                    
    def apply_recourse(self, dataset, factuals, generator, generator_name='Wachter'):
        """
        Generate (a set of) counterfactual explanations with a provided generator.
        
        Args:
            dataset (DataCatalog): 
                Catalog containing a dataframe, set of train and test records, and the target.
            factuals (pandas.DataFrame): 
                One or more records from a dataset used to train the black-box model.
            generator (RecourseMethod): 
                Generator that finds counterfactual explanations using a black-box model.
            generator_name (str):
                Name of the generator that is used to apply recourse.
                
        Returns:
            int: Number of successfully generated counterfactuals.
        """
        log.info(f"Applying the {generator_name} generator.")
        
        if factuals is None or len(factuals) == 0:
            return 0
                                              
        counterfactuals = generator.get_counterfactuals(factuals).dropna()
        dataset._df.update(counterfactuals)
        
        return len(counterfactuals.index)
  
                              
    def apply_recourse_with_timeout(self, dataset, factuals, generator, generator_name='DICE'):
        """
        Generate (a set of) counterfactual explanations with a provided generator. 
        These explanations are applied one-by-one with a specific timeout for every single factual.
        
        Args:
            dataset (DataCatalog): 
                Catalog containing a dataframe, set of train and test records, and the target.
            factuals (pandas.DataFrame): 
                One or more records from a dataset used to train the black-box model.
            generator (RecourseMethod): 
                Generator that finds counterfactual explanations using a black-box model.
            generator_name (str):
                Name of the generator that is used to apply recourse.
                
        Returns:
            int: Number of successfully generated counterfactuals.
        """
        log.info(f"Applying the {generator_name} generator.")
        
        if factuals is None or len(factuals) == 0:
            return 0
                              
        counterfactuals_found = 0
        for i in range(len(factuals)):
            f = factuals.iloc[[i]]
            
            log.info(f"Generating counterfactual {i + 1} with {generator_name}")
            # CARLA does not implement a timeout for the generators by default
            # but we want to prevent the code from running indefinitely
            counterfactuals = recourse_controller(recourse_worker, self.generator_timeout, generator, f)
            # We only want to overwrite the existing data if counterfactual generation was successful
            if counterfactuals is not None and not counterfactuals.empty:
                dataset._df.iloc[f.index[0]] = counterfactuals.iloc[0]
                counterfactuals_found += 1
                         
        return counterfactuals_found
                
                         
    def save_data(self, path=None):
        """
        Write the data collected throughout the experiment into a .json file.
        
        Args:
            path (str):
                Directory where the dictionary of experiment data should be written.
        """
        
        path = path or f'../experiment_data/{self.experiment_name}/measurements.json'
        with open(path, 'w') as outfile:
            json.dump(self.experiment_data, outfile, sort_keys=True, indent=4)

In [6]:
def recourse_worker(generator, factual):
    """
    Apply algorithmic recourse for a (set of) factuals using a chosen generator.
    
    Args:
        generator (RecourseMethod): 
            Generator that finds counterfactual explanations using a black-box model.
        factual (pandas.DataFrame): 
            One or more records from a dataset used to train the black-box model.
        
    Returns:
        pandas.DataFrame: A counterfactual explanation for the provided factual.
    """
    if factual is None:
        raise ValueException('Provided with a non-existent factual')
        return None
    
    counterfactuals = generator.get_counterfactuals(factual).dropna()
    if not counterfactuals.empty:
        return counterfactuals.sample().astype(float)
    raise FunctionTimedOut()
    
def recourse_controller(function, max_wait_time, generator, factual):
    """
    Wrapper function that ensures the application of recourse does not run indefinitely.
    
    Args:
        function (Callable): 
            Function that will have its execution placed under a timeout.
        max_wait_time (int): 
            Number of seconds after which the `function` will time out.
        generator (RecourseMethod): 
            Generator that finds counterfactual explanations using a black-box model.
        factual (pandas.DataFrame): 
            One or more records from a dataset used to train the black-box model.
        
    Returns:
        pandas.DataFrame: A counterfactual explanation for the provided factual if found.
    """
    try:
        return func_timeout(max_wait_time, function, args=[generator, factual]) 
    except FunctionTimedOut:
        log.info("Timeout - No Counterfactual Explanation Found")

    return None

In [7]:
def train_model(dataset, hyper_params, model_type='ann', retrain=False):
    """
    Instantiates and trains a black-box model within a CARLA wrapper that will be used to generate explanations.
    
    Args:
        dataset (DataCatalog): 
            Catalog containing a dataframe, set of train and test records, and the target.
        hyper_params (dict): 
            Dictionary storing all custom hyper-parameter values for the model.
        
    Returns:
        MLModelCatalog: 
            Classifier with additional utilities required by CARLA.
    """
    kwargs = {'save_name_params': "_".join([str(size) for size in hyper_params['hidden_size']])}
    model = DynamicMLModelCatalog(dataset, model_type=model_type,
                                  load_online=False, backend="pytorch", **kwargs)

    # force_train is enabled to ensure that the model is not reused from the cache
    if not retrain:
        log.info("Training the initial model")
        model.train(learning_rate=hyper_params['learning_rate'],
                    epochs=hyper_params['epochs'],
                    batch_size=hyper_params['batch_size'],
                    hidden_size=hyper_params['hidden_size'],
                    force_train=True)
    else:
        model.retrain(learning_rate=hyper_params['learning_rate'],
                      epochs=hyper_params['epochs'],
                      batch_size=hyper_params['batch_size'])
    
    return model